Сначала загрузим данные и посмотрим на колонки

In [1]:
import pandas as pd

pandas_df = pd.read_csv("./исходные данные/MOCK_DATA (1).csv")
pandas_df.columns

Index(['id', 'customer_first_name', 'customer_last_name', 'customer_age',
       'customer_email', 'customer_country', 'customer_postal_code',
       'customer_pet_type', 'customer_pet_name', 'customer_pet_breed',
       'seller_first_name', 'seller_last_name', 'seller_email',
       'seller_country', 'seller_postal_code', 'product_name',
       'product_category', 'product_price', 'product_quantity', 'sale_date',
       'sale_customer_id', 'sale_seller_id', 'sale_product_id',
       'sale_quantity', 'sale_total_price', 'store_name', 'store_location',
       'store_city', 'store_state', 'store_country', 'store_phone',
       'store_email', 'pet_category', 'product_weight', 'product_color',
       'product_size', 'product_brand', 'product_material',
       'product_description', 'product_rating', 'product_reviews',
       'product_release_date', 'product_expiry_date', 'supplier_name',
       'supplier_contact', 'supplier_email', 'supplier_phone',
       'supplier_address', 'supplier_cit

Из названия колонок можно сразу выделить сущности, которые будут в звезде:

1. customer
2. customer_pet
3. seller
4. product
5. sale
6. store
7. supplier

Можно заметить, что тут есть колонки, где используется внешний ключ(sale_product_id, например). Получается, что это внешний ключ на строку исходной таблица, а не на конкретную сущность. Из этого можно сделать вывод, что при заполнении снежинки на каждой строке будут либо создаваться новые, либо переписываться старые записи с новым идентификатором строки. Причем во всех файлах MOCK_DATA... id начинается с 1, так что надо будет к id прибавлять количество уже вставленных записей

In [2]:
for col in filter(lambda x: 'id' in x, pandas_df.columns):
    print(col, len(pandas_df[pandas_df["id"] != pandas_df[col]]))

id 0
sale_customer_id 0
sale_seller_id 0
sale_product_id 0


Да, все поля с id совпадают с id строки, так что можно вставлять строки как есть

In [3]:
inserted_rows = 0

In [4]:
for col in pandas_df.columns:
    unique = pandas_df[col].unique()
    if len(unique) > 200 or 'desc' in col:
        continue
    print(f"{col}: {unique}")

customer_age: [63 26 47 81 72 42 80 77 57 82 31 73 32 30 58 54 46 76 19 61 39 67 45 40
 27 18 41 38 55 78 62 70 64 36 43 83 33 28 69 51 85 71 37 79 23 35 20 65
 21 24 66 75 84 49 34 52 59 68 74 22 53 60 44 25 56 50 29 48]
customer_country: ['France' 'Philippines' 'Madagascar' 'Portugal' 'Russia' 'China' 'Albania'
 'Venezuela' 'United States' 'Thailand' 'Japan' 'Greece' 'Tanzania' 'Peru'
 'Indonesia' 'Cuba' 'Poland' 'Brazil' 'Sweden' 'Czech Republic' 'Haiti'
 'Pakistan' 'Montenegro' 'Macedonia' 'Israel' 'Honduras' 'Colombia'
 'Kyrgyzstan' 'Zimbabwe' 'Lebanon' 'Argentina' 'Bulgaria' 'Yemen'
 'Afghanistan' 'Mali' 'Mexico' 'Mauritania'
 'Democratic Republic of the Congo' 'Mauritius' 'Falkland Islands'
 'Kazakhstan' 'Ukraine' 'Sierra Leone' 'Spain' 'Dominican Republic'
 'Palestinian Territory' 'Syria' 'Vietnam' 'Cameroon' 'Angola' 'Egypt'
 'Costa Rica' 'Serbia' 'Kenya' 'Iran' 'Malaysia' 'Italy' 'Chile' 'Nepal'
 'Ethiopia' 'Croatia' 'Paraguay' 'Luxembourg' 'South Africa' 'Germany'
 'French P

In [5]:
pd.to_datetime(pandas_df['product_expiry_date'].head())

0   2026-02-28
1   2029-09-11
2   2024-01-05
3   2025-06-03
4   2030-07-10
Name: product_expiry_date, dtype: datetime64[ns]

In [6]:
pd.DataFrame({'a': pd.Series(dtype='datetime64[ns]'), 'b': pd.Series(dtype="int"), "c": pd.Series(dtype="float"), "d" : pd.Series(dtype="str")}).dtypes

a    datetime64[ns]
b             int64
c           float64
d            object
dtype: object

Терерь я опишу функцию, которая будет разделать большой фрейм на несколько меньших

In [7]:
inserted_rows = 0
customer = pd.DataFrame({
    "customer_first_name": pd.Series(dtype="str"),
    'customer_last_name': pd.Series(dtype="str"),
    'customer_age': pd.Series(dtype="int"),
    'customer_email': pd.Series(dtype="str"),
    'customer_country': pd.Series(dtype="str"),
    'customer_postal_code' : pd.Series(dtype="str"),
    'customer_pet_id' : pd.Series(dtype="int")
})
pet = pd.DataFrame({
    "customer_pet_type": pd.Series(dtype="str"),
    "customer_pet_name": pd.Series(dtype="str"),
    "customer_pet_breed": pd.Series(dtype="str"),
})
seller = pd.DataFrame({
    'seller_first_name': pd.Series(dtype="str"),
    'seller_last_name': pd.Series(dtype="str"),
    'seller_email': pd.Series(dtype="str"),
    'seller_country': pd.Series(dtype="str"),
    'seller_postal_code': pd.Series(dtype="str")
})
product = pd.DataFrame({
    'product_name': pd.Series(dtype="str"),
    'product_category': pd.Series(dtype="str"),
    'product_price': pd.Series(dtype="float"),
    'product_quantity': pd.Series(dtype="float"),
    'product_weight': pd.Series(dtype="float"),
    'product_color': pd.Series(dtype="str"),
    'product_size': pd.Series(dtype="str"),
    'product_brand': pd.Series(dtype="str"),
    'product_material': pd.Series(dtype="str"),
    'product_description': pd.Series(dtype="str"),
    'product_rating': pd.Series(dtype="float"),
    'product_reviews': pd.Series(dtype="int"),
    'product_release_date': pd.Series(dtype="datetime64[ns]"),
    'product_expiry_date': pd.Series(dtype="datetime64[ns]"),
    'pet_category' : pd.Series(dtype="str")
})
sale = pd.DataFrame({
    'sale_date': pd.Series(dtype="datetime64[ns]"),
    'sale_customer_id': pd.Series(dtype="int"),
    'sale_seller_id': pd.Series(dtype="int"),
    'sale_store_id': pd.Series(dtype="int"),
    'sale_quantity': pd.Series(dtype="float"),
    'sale_total_price': pd.Series(dtype="float"),
    'sale_supplier_id': pd.Series(dtype="int"),
    'sale_product_id': pd.Series(dtype="int"),
})
store = pd.DataFrame({
    'store_name': pd.Series(dtype="str"),
    'store_location': pd.Series(dtype="str"),
    'store_city': pd.Series(dtype="str"),
    'store_state': pd.Series(dtype="str"),
    'store_country': pd.Series(dtype="str"),
    'store_phone': pd.Series(dtype="str"),
    'store_email': pd.Series(dtype="str"),
})

supplier = pd.DataFrame({
    'supplier_name': pd.Series(dtype="str"),
    'supplier_contact': pd.Series(dtype="str"),
    'supplier_email': pd.Series(dtype="str"),
    'supplier_phone': pd.Series(dtype="str"),
    'supplier_address': pd.Series(dtype="str"),
    'supplier_city': pd.Series(dtype="str"),
    'supplier_country': pd.Series(dtype="str")
})

In [8]:
def split_raw_df(df):
    global customer, pet, seller, product, sale, store, supplier, inserted_rows
    
    customer = pd.concat([df, customer], join='inner', ignore_index='true')
    customer.index += 1
    customer["pet_id"] = customer.index
    pet = pd.concat([df, pet], join='inner', ignore_index='true')
    pet.index += 1
    seller = pd.concat([df, seller], join='inner', ignore_index='true')
    seller.index += 1
    product = pd.concat([df, product], join='inner', ignore_index='true')
    product.index += 1
    sale = pd.concat([df, sale], join='inner', ignore_index='true')
    sale.index += 1
    sale["sale_store_id"] = sale.index
    sale["sale_supplier_id"] = sale.index
    store = pd.concat([df, store], join='inner', ignore_index='true')
    store.index += 1
    supplier = pd.concat([df, supplier], join='inner', ignore_index='true')
    supplier.index += 1

    inserted_rows += len(df)

In [9]:
import os
dir, _, files = next(iter(os.walk("./исходные данные/")))
for file in files:
    pandas_df = pd.read_csv(os.path.join(dir, file))
    split_raw_df(pandas_df)

In [10]:
customer.columns = [ 'first_name', 'last_name', 'age', 'email', 'country', 'postal_code' , 'pet_id' ]
pet.columns = [ "type", "name", "breed" ]
seller.columns = [ 'first_name', 'last_name', 'email', 'country', 'postal_code' ]
product.columns = ['name', 'category', 'price', 'quantity', 'pet_category', 'weight', 'color', 'size', 'brand', 'material', 'description', 'rating', 'reviews', 'release_date', 'expiry_date' ]
sale.columns = [ 'date', 'customer_id', 'seller_id', 'store_id',  'quantity', 'total_price', 'supplier_id','product_id' ]
store.columns = [ 'name', 'location', 'city', 'state', 'country', 'phone', 'email' ]
supplier.columns = [ 'name', 'contact', 'email', 'phone', 'address', 'city', 'country' ]


In [11]:
product["expiry_date"] = product["expiry_date"].astype("datetime64[ns]")
product["release_date"] = product["release_date"].astype("datetime64[ns]")
sale["date"] = sale["date"].astype("datetime64[ns]")

In [12]:
customer.head()

,first_name,last_name,age,email,country,postal_code,pet_id
1,Consolata,Dufoure,21,awigginton0@wordpress.com,China,NaN,1
2,Gaylord,Sawter,47,delsdon1@soundcloud.com,Madagascar,NaN,2
3,Nicky,Lattie,74,gcoupman2@bigcartel.com,Mexico,29020,3
4,Lusa,Ruler,24,mpatise3@craigslist.org,China,NaN,4
5,Darb,Scimonelli,23,rsheriff4@webs.com,China,NaN,5


In [13]:
sale.head()

,date,customer_id,seller_id,store_id,quantity,total_price,supplier_id,product_id
1,2021-08-01,1,1,1,6.0,54.36,1,1
2,2021-10-08,2,2,2,1.0,373.00,2,2
3,2021-02-11,3,3,3,9.0,499.62,3,3
4,2021-07-04,4,4,4,5.0,461.35,4,4
5,2021-10-21,5,5,5,3.0,92.46,5,5


Теперь можно подключить spark и сделать экспорт данных в postgres. Я заметил нюанс, который скорее всего связан с jupyter notebook: когда создается переменная spark ниже, то потом при пересоздании он использует старую версию переменной, т.е. если в конфиге указать только jar файл для postgresql, а в другой ячейке для postgresql и clickhouse, то при подключении к clickhouse будет ошибка, типа jar файл не найден

Поэтому сразу подключу оба jar файла

Можно заметить, что для работы spark нужно создавать сессии, так что имеет смысл выделить создание и закрытие сессии в отдельную функцию-декоратор

In [14]:
from functools import wraps
from pyspark.sql import SparkSession

def spark_session(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        spark = SparkSession.builder \
            .appName("Spark SQL with PostgreSQL and ClickHouse") \
            .config("spark.jars", "postgresql-42.6.0.jar,clickhouse-jdbc-0.8.5-all.jar")\
            .getOrCreate()
        result = func(*args, spark=spark, **kwargs)
        spark.stop()
        return result
    return wrapper

dbData = {
    "postgresql" :  ("jdbc:postgresql://postgres:5432/spark_db", {
        "user": "spark_user",
        "password": "spark_password",
        "driver": "org.postgresql.Driver"
    }),
    "clickhouse" : ("jdbc:clickhouse://clickhouse:8123/default", {
        "driver": "com.clickhouse.jdbc.ClickHouseDriver",
        "user": "custom_user",
        "password": "custom_password"
    })
}

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

@spark_session
def view_preview(spark):
    data = [
    	("pets", spark.createDataFrame(pet)),
    	("customers", spark.createDataFrame(customer)),
    	("sellers", spark.createDataFrame(seller)),
    	("products", spark.createDataFrame(product)),
    	("stores", spark.createDataFrame(store)),
    	("suppliers", spark.createDataFrame(supplier)),
        ("sales", spark.createDataFrame(sale))
    ]
    data[6][1].createOrReplaceTempView("sales")
    sqlDf = spark.sql("select * from sales limit 10;")
    sqlDf.show()
view_preview()

+-------------------+-----------+---------+--------+--------+-----------+-----------+----------+
|               date|customer_id|seller_id|store_id|quantity|total_price|supplier_id|product_id|
+-------------------+-----------+---------+--------+--------+-----------+-----------+----------+
|2021-08-01 00:00:00|          1|        1|       1|     6.0|      54.36|          1|         1|
|2021-10-08 00:00:00|          2|        2|       2|     1.0|      373.0|          2|         2|
|2021-02-11 00:00:00|          3|        3|       3|     9.0|     499.62|          3|         3|
|2021-07-04 00:00:00|          4|        4|       4|     5.0|     461.35|          4|         4|
|2021-10-21 00:00:00|          5|        5|       5|     3.0|      92.46|          5|         5|
|2021-10-13 00:00:00|          6|        6|       6|     6.0|     148.63|          6|         6|
|2021-09-23 00:00:00|          7|        7|       7|     3.0|     446.88|          7|         7|
|2021-07-16 00:00:00|         

Экспорт данных в postgresql

In [16]:
@spark_session
def export_to_postgresql(spark):
    data = [
    	("pets", spark.createDataFrame(pet)),
    	("customers", spark.createDataFrame(customer)),
    	("sellers", spark.createDataFrame(seller)),
    	("products", spark.createDataFrame(product)),
    	("stores", spark.createDataFrame(store)),
    	("suppliers", spark.createDataFrame(supplier)),
        ("sales", spark.createDataFrame(sale))
    ]
    pg_url, pg_props = dbData["postgresql"]
    
    for table, df in data:
        df.write\
        .jdbc(url=pg_url, table=table, mode="append", properties=pg_props)

export_to_postgresql()

Теперь будем собирать статистику и делать витрины

Витрина на продукты

In [17]:
from pyspark.sql.types import StructType, StructField, StringType, MapType, FloatType, ArrayType, Row
from pyspark.sql.functions import create_map, array_agg, array, map_from_entries, struct

@spark_session
def products_report(spark):
    pg_url, pg_props =  dbData["postgresql"]
    
    products = spark.read.jdbc(url=pg_url, table="products", properties=pg_props)
    sales = spark.read.jdbc(url=pg_url, table="sales", properties=pg_props)
    
    sales.createOrReplaceTempView("sales")
    products.createOrReplaceTempView("products")
    
    most_salling_products = spark.sql("""
    select array_agg(t1.name) as names
    from (
    select t1.name as name, count(t2.product_id) as sales
    from
        products as t1
    inner join 
        sales as t2
    on t1.id = t2.product_id
    group by t1.name
    order by sales desc
    limit 10) as t1;
    """)
    
    revenue_per_category = spark.sql(
    """
    select struct(t1.category, sum(price)) as data
        from 
        products as t1
    inner join 
        sales as t2
    on t1.id = t2.product_id
    group by t1.category;
    """)
    
    mean_mark_and_reviews_per_product = spark.sql("""
    select
        name,
        avg(rating) as mean_avg,
        avg(reviews) as mean_reviews
    from products
    group by name;
    """)
    
    print("Самые продаваемые товары")
    display(most_salling_products.toPandas())
    
    print("Выручка по категориям")
    display(revenue_per_category.toPandas())
    
    print("Средняя оценка и количество комментариев для каждого предмета(первые 5)")
    display(mean_mark_and_reviews_per_product.toPandas().head())
    
    schema = StructType([
        StructField("mostRelevantProducts", ArrayType(StringType()), False),
        StructField("categoryRevenue", MapType(StringType(), FloatType()), True),
        StructField("ratePerProduct", MapType(StringType(), FloatType()), True),
        StructField("commentPerProduct", MapType(StringType(), FloatType()), True),
    ])
    
    most_relevant = most_salling_products
    category_revenue = revenue_per_category.select(map_from_entries(array_agg("data")).alias("data"))
    marks_per_product = mean_mark_and_reviews_per_product.select(map_from_entries(array_agg(struct("name", "mean_avg"))).alias("data"))
    reviews_per_product = mean_mark_and_reviews_per_product.select(map_from_entries(array_agg(struct("name","mean_reviews"))).alias("data"))
    
    ch_products_df = spark.createDataFrame(data=[
        (most_relevant.first()["names"],
         category_revenue.first()["data"],
         marks_per_product.first()["data"],
         reviews_per_product.first()["data"])
    ], schema=schema)
    
    ch_products_df2 = spark.createDataFrame(data=[
        Row(data=ch_products_df.toJSON().first())
    ])
    
    ch_url, ch_props =  dbData["clickhouse"]
    
    ch_products_df2.write.jdbc(url=ch_url, table="products", mode="append", properties=ch_props)

products_report()

Самые продаваемые товары


,names
0,"[Dog Food, Bird Cage, Cat Toy]"


Выручка по категориям


,data
0,"(Cage, 217795.48999999982)"
1,"(Food, 202240.4700000004)"
2,"(Toy, 217933.7900000001)"


Средняя оценка и количество комментариев для каждого предмета(первые 5)


,name,mean_avg,mean_reviews
0,Bird Cage,3.000149,501.867542
1,Dog Food,3.018299,495.182090
2,Cat Toy,3.006860,506.564521


Витрина на продажи по покупателям

In [18]:
from pyspark.sql.types import StructType, StructField, StringType, MapType, FloatType, ArrayType, Row, IntegerType
from pyspark.sql.functions import create_map, array_agg, array, map_from_entries, struct

@spark_session
def customers_report(spark):
    pg_url, pg_props =  dbData["postgresql"]
    
    customers = spark.read.jdbc(url=pg_url, table="customers", properties=pg_props)
    sales = spark.read.jdbc(url=pg_url, table="sales", properties=pg_props)
    
    sales.createOrReplaceTempView("sales")
    customers.createOrReplaceTempView("customers")
    
    most_relevant_customers = spark.sql("""
    select array_agg(t1.name) as names
    from (
    select t1.first_name as name, count(t1.id) as sales
    from
        customers as t1
    inner join 
        sales as t2
    on t1.id = t2.customer_id
    group by t1.first_name
    order by sales desc
    limit 10) as t1;
    """)
    
    customer_country_stratification = spark.sql(
    """
		select array_agg(struct(t1.*)) as entries from (
		select t1.country as country, count(t1.id) as sales
			from 
					customers as t1
			inner join 
					sales as t2
			on t1.id = t2.customer_id
			group by t1.country
		) as t1;
    """)
    
    mean_order_cost_per_customer = spark.sql("""
		select array_agg(struct(t1.*)) as entries
    from (
		select
        t1.first_name as name,
        avg(t2.total_price) as mean_order_cost
		from 
				customers as t1
		inner join 
				sales as t2
    group by t1.first_name
		) as t1;
    """)
    
    print("Самые покупающие покупатели")
    display(most_relevant_customers.toPandas())
    
    print("Распределение продаж по странам")
    display(customer_country_stratification.toPandas())
    
    print("Средняя стоимость покупок на пользователя(первые 5)")
    display(mean_order_cost_per_customer.toPandas().head())
    
    schema = StructType([
        StructField("mostRelevantCustomers", ArrayType(StringType()), False),
        StructField("customerCountryStratification", MapType(StringType(), IntegerType()), False),
        StructField("meanOrderCostPerCustomer", MapType(StringType(), FloatType()), False),
    ])
    
    most_relevant_customers = most_relevant_customers.first()["names"]
    customer_country_stratification = customer_country_stratification.select(map_from_entries("entries").alias("data")).first()["data"]
    mean_order_cost_per_customer = mean_order_cost_per_customer.select(map_from_entries("entries").alias("data")).first()["data"]
    
    ch_products_df = spark.createDataFrame(data=[
        (most_relevant_customers,
         customer_country_stratification,
         mean_order_cost_per_customer)
    ], schema=schema)
    
    ch_products_df2 = spark.createDataFrame(data=[
        Row(data=ch_products_df.toJSON().first())
    ])
    
    ch_url, ch_props =  dbData["clickhouse"]
    
    ch_products_df2.write.jdbc(url=ch_url, table="sales_per_customers", mode="append", properties=ch_props)

customers_report()

Самые покупающие покупатели


,names
0,"[Sibyl, Sybyl, Konrad, Georas, Allyce, Land, F..."


Распределение продаж по странам


,entries
0,"[(Chad, 2), (Russia, 692), (Paraguay, 25), (Ye..."


Средняя стоимость покупок на пользователя(первые 5)


,entries
0,"[(Reta, 264.1901656132493), (Donielle, 264.190..."


Распределение продаж по времени

In [19]:
from pyspark.sql.types import StructType, StructField, StringType, MapType, FloatType, ArrayType, Row, IntegerType, Tuple
from pyspark.sql.functions import create_map, array_agg, array, map_from_entries, struct, col

@spark_session
def sales_per_time_report(spark):
    pg_url, pg_props =  dbData["postgresql"]
    
    sales = spark.read.jdbc(url=pg_url, table="sales", properties=pg_props)
    
    sales.createOrReplaceTempView("sales")
    
    year_and_month_trands = spark.sql("""
		select cast(t1.year_month as varchar(20)) as year_month, t1.sales - lag(t1.sales) over (order by t1.year_month) as trand
		from 
          (select
              t1.year_month,
              count(*) as sales
            from (
        		select 
        			date_trunc('month', date) as year_month
        		from sales
    		) as t1
            group by t1.year_month
            order by t1.year_month) as t1;
    """)
    
    period_revenue_comparison = spark.sql(
    """
		select 
          t1.period,
          sum(t1.total_price) as overall_price
        from (
        select 
			date_trunc('month', date) as period,
            total_price
		from sales
        ) as t1
        group by t1.period
        order by t1.period;
    """)
    
    mean_order_size_per_month = spark.sql("""
			select
				t1.month as month,
				avg(t1.order_size) as mean_order_size
			from (
				select
					extract (month from date) as month,
					quantity as order_size
				from sales
			) as t1
            group by t1.month;
    """)
    
    print("Тренды по годам и месяцам")
    display(year_and_month_trands.toPandas())
    
    print("Сравнение выручки по периодам(месяцам)")
    display(period_revenue_comparison.toPandas())
    
    print("Средний размер покупок за месяц")
    display(mean_order_size_per_month.toPandas().head())
    
    schema = StructType([
        StructField("yearAndMonthTrands", MapType(StringType(), FloatType()), False),
        StructField("periodsRevenueComparison", MapType(StringType(), FloatType()), False),
        StructField("meanOrderSizePerMonth", MapType(StringType(), FloatType()), False),
    ])
    
    
    ch_products_df = spark.createDataFrame(data=[
        (year_and_month_trands.select(map_from_entries(array_agg(struct("year_month", col("trand").cast("float")))).alias("data")).first()["data"],
         period_revenue_comparison.select(map_from_entries(array_agg(struct("period", "overall_price"))).alias("data")).first()["data"],
         mean_order_size_per_month.select(map_from_entries(array_agg(struct("month", "mean_order_size"))).alias("data")).first()["data"])
    ], schema=schema)
    
    ch_products_df2 = spark.createDataFrame(data=[
        Row(data=ch_products_df.toJSON().first())
    ])
    
    ch_url, ch_props =  dbData["clickhouse"]
    
    ch_products_df2.write.jdbc(url=ch_url, table="sales_per_times", mode="append", properties=ch_props)

sales_per_time_report()


Тренды по годам и месяцам


,year_month,trand
0,2021-01-01 00:00:00,NaN
1,2021-02-01 00:00:00,-165.0
2,2021-03-01 00:00:00,120.0
3,2021-04-01 00:00:00,-24.0
4,2021-05-01 00:00:00,-14.0
5,2021-06-01 00:00:00,4.0
6,2021-07-01 00:00:00,18.0
7,2021-08-01 00:00:00,78.0
8,2021-09-01 00:00:00,-54.0
9,2021-10-01 00:00:00,72.0


Сравнение выручки по периодам(месяцам)


,period,overall_price
0,2021-01-01,300570.51
1,2021-02-01,254191.91
2,2021-03-01,273299.26
3,2021-04-01,268026.29
4,2021-05-01,268367.77
5,2021-06-01,276336.76
6,2021-07-01,276384.56
7,2021-08-01,290565.26
8,2021-09-01,282355.07
9,2021-10-01,299655.53


Средний размер покупок за месяц


,month,mean_order_size
0,12,32.552659
1,1,36.762943
2,6,33.487280
3,3,35.450758
4,5,25.666012


Распределение продаж по магазинам

In [20]:
from pyspark.sql.types import StructType, StructField, StringType, MapType, FloatType, ArrayType, Row, IntegerType, Tuple
from pyspark.sql.functions import create_map, array_agg, array, map_from_entries, struct, col

@spark_session
def sales_per_stores_report(spark):
		pg_url, pg_props =	dbData["postgresql"]
		
		sales = spark.read.jdbc(url=pg_url, table="sales", properties=pg_props)
		stores = spark.read.jdbc(url=pg_url, table="stores", properties=pg_props)
		
		sales.createOrReplaceTempView("sales")
		stores.createOrReplaceTempView("stores")
		
		most_relevant_stores = spark.sql("""
		select t1.name, count(*) as sales
		from 
			sales as t2
		inner join
			stores as t1
		on t2.store_id = t1.id
		group by t1.name
        order by sales desc
		""")
		
		state_country_sales_distribution = spark.sql(
		"""
		select concat(t1.country, ',', t1.state) as location, count(*) as sales
		from 
			sales as t2
		inner join
			stores as t1
		on t2.store_id = t1.id
		group by t1.country, t1.state
		""")
		
		mean_order_cost_per_store = spark.sql("""
		select t1.name, avg(t2.total_price) as mean_order_cost
        from
			sales as t2
		inner join
			stores as t1
		group by t1.name;
		""")
		
		print("Лучшие магазины")
		display(most_relevant_stores.toPandas())
		
		print("Распределение по странам и городам")
		display(state_country_sales_distribution.toPandas())
		
		print("Средняя стоимость заказа в магазине")
		display(mean_order_cost_per_store.toPandas())
		
		schema = StructType([
				StructField("mostRelevantStores", ArrayType(StringType()), False),
				StructField("stateCountrySalesDistribution", MapType(StringType(), IntegerType()), False),
				StructField("meanOrderCostPerStore", MapType(StringType(), FloatType()), False),
		])
		
		
		ch_products_df = spark.createDataFrame(data=[
				(most_relevant_stores.select(array_agg("name").alias("data")).first()["data"],
				 state_country_sales_distribution.select(map_from_entries(array_agg(struct("location", "sales"))).alias("data")).first()["data"],
				 mean_order_cost_per_store.select(map_from_entries(array_agg(struct("name", "mean_order_cost"))).alias("data")).first()["data"])
		], schema=schema)
		
		ch_products_df2 = spark.createDataFrame(data=[
				Row(data=ch_products_df.toJSON().first())
		])
		
		ch_url, ch_props =	dbData["clickhouse"]
		
		ch_products_df2.write.jdbc(url=ch_url, table="sales_per_stores", mode="append", properties=ch_props)

sales_per_stores_report()


Лучшие магазины


,name,sales
0,Skimia,98
1,Lajo,92
2,Quatz,83
3,Skyble,83
4,Quamba,82
...,...,...
378,Latz,3
379,Vinte,2
380,Dabshots,2
381,Skivee,2


Распределение по странам и городам


,location,sales
0,"Canada,TX",11
1,"Georgia,NaN",13
2,"Russia,C1",2
3,"Kyrgyzstan,NaN",3
4,"France,18",1
...,...,...
512,"Mali,NaN",2
513,"China,16",1
514,"Brazil,03",5
515,"Philippines,12",12


Средняя стоимость заказа в магазине


,name,mean_order_cost
0,Jetwire,264.190166
1,Jaxworks,264.190166
2,Brainlounge,264.190166
3,Snaptags,264.190166
4,Reallinks,264.190166
...,...,...
378,Thoughtbridge,264.190166
379,Skyble,264.190166
380,Rhyzio,264.190166
381,Yamia,264.190166


Распреление продаж по диллерам

In [21]:
from pyspark.sql.types import StructType, StructField, StringType, MapType, FloatType, ArrayType, Row, IntegerType, Tuple
from pyspark.sql.functions import create_map, array_agg, array, map_from_entries, struct, col

@spark_session
def sales_per_suppliers_report(spark):
		pg_url, pg_props =	dbData["postgresql"]
		
		sales = spark.read.jdbc(url=pg_url, table="sales", properties=pg_props)
		suppliers = spark.read.jdbc(url=pg_url, table="suppliers", properties=pg_props)
		products = spark.read.jdbc(url=pg_url, table="products", properties=pg_props)
		
		sales.createOrReplaceTempView("sales")
		suppliers.createOrReplaceTempView("suppliers")
		products.createOrReplaceTempView("products")
		
		most_relevant_suppliers = spark.sql("""
		select t1.name, count(*) as sales
		from 
			sales as t2
		inner join
			suppliers as t1
		on t2.supplier_id = t1.id
		group by t1.name
		order by sales desc
		""")
		
		mean_product_cost_per_supplier = spark.sql(
		"""
		select t3.supplier_name, avg(t4.price) as mean_product_price
		from 
			(
			select t1.name as supplier_name, t2.* from
			sales as t2
		inner join
			suppliers as t1
		on t2.supplier_id = t1.id
			) as t3
		inner join
			products as t4
		on t3.product_id = t4.id
		group by t3.supplier_name;
		""")
		
		supplier_sales_per_country_distribution = spark.sql("""
		select t1.country, count(*) as sales
		from
			sales as t2
		inner join
			suppliers as t1
		on t2.supplier_id = t1.id
		group by t1.country;
		""")
		
		print("Самые крутые диллеры")
		display(most_relevant_suppliers.toPandas())
		
		print("Средняя цена товара на диллера")
		display(mean_product_cost_per_supplier.toPandas())
		
		print("Распределение диллеров по странам")
		display(supplier_sales_per_country_distribution.toPandas().head())
		
		schema = StructType([
				StructField("mostRelevantSuppliers", ArrayType(StringType()), False),
				StructField("meanProductCostPerSupplier", MapType(StringType(), FloatType()), False),
				StructField("supplierSalesPerCountryDistribution", MapType(StringType(), IntegerType()), False),
		])
		
		
		ch_products_df = spark.createDataFrame(data=[
				(most_relevant_suppliers.select(array_agg("name").alias("data")).first()["data"],
				 mean_product_cost_per_supplier.select(map_from_entries(array_agg(struct("supplier_name", "mean_product_price"))).alias("data")).first()["data"],
				 supplier_sales_per_country_distribution.select(map_from_entries(array_agg(struct("country", "sales"))).alias("data")).first()["data"])
		], schema=schema)
		
		ch_products_df2 = spark.createDataFrame(data=[
				Row(data=ch_products_df.toJSON().first())
		])
		
		ch_url, ch_props =	dbData["clickhouse"]
		
		ch_products_df2.write.jdbc(url=ch_url, table="sales_per_suppliers", mode="append", properties=ch_props)

sales_per_suppliers_report()

Самые крутые диллеры


,name,sales
0,Brightdog,126
1,Feedmix,123
2,Youfeed,114
3,Dynazzy,112
4,Oloo,111
...,...,...
378,Gabtune,16
379,Yambee,16
380,Minyx,15
381,DabZ,13


Средняя цена товара на диллера


,supplier_name,mean_product_price
0,Jetwire,47.713913
1,Jaxworks,42.213125
2,Brainlounge,48.400909
3,Reallinks,56.333125
4,Snaptags,44.457931
...,...,...
378,Skyble,50.837778
379,Thoughtbridge,56.454000
380,Rhyzio,50.361481
381,Yamia,57.033571


Распределение диллеров по странам


,country,sales
0,Chad,8
1,Russia,687
2,Paraguay,14
3,Yemen,48
4,Senegal,15


Распределение качества товаров

In [22]:
from pyspark.sql.types import StructType, StructField, StringType, MapType, FloatType, ArrayType, Row, IntegerType, Tuple
from pyspark.sql.functions import create_map, array_agg, array, create_map, struct, corr

@spark_session
def quantity_of_products_report(spark):
		pg_url, pg_props =	dbData["postgresql"]
		
		sales = spark.read.jdbc(url=pg_url, table="sales", properties=pg_props)
		products = spark.read.jdbc(url=pg_url, table="products", properties=pg_props)
		
		sales.createOrReplaceTempView("sales")
		products.createOrReplaceTempView("products")
		
		products_highest_rate = spark.sql("""
		select name as key, rating as value
		from (
            select name, rating
            from products
            order by rating desc
            limit 10
        );
		""")
		
		product_lowest_rate = spark.sql("""
		select name as key, rating as value
		from (
            select name, rating
            from products
            order by rating
            limit 10
        );
		""")
		
		rate_and_sales_count_correlation = spark.sql("""
		select rating, count(*) as sales
		from
			sales as t2
		inner join
			products as t1
		on t2.product_id = t1.id
		group by t1.rating;
		""")

		products_with_biggest_coments_quantity = spark.sql("""
		select name as key, reviews as value
		from (
            select name, reviews
            from products
            order by reviews desc
            limit 25
        );
		""")
		
		print("Самые крутые товары")
		display(products_highest_rate.toPandas())
		
		print("Самые плохие товары")
		display(product_lowest_rate.toPandas())
		
		print("Корреляция продаж и рейтинга(сырые данные)")
		display(rate_and_sales_count_correlation.toPandas().head())

		print("Товары с наибольшим количеством комментариев")
		display(products_with_biggest_coments_quantity.toPandas())

		
		schema = StructType([
				StructField("productsHighestRate", ArrayType(MapType(StringType(), FloatType())), False),
				StructField("productsLowestRate", ArrayType(MapType(StringType(), FloatType())), False),
				StructField("rateAndSalesCountCorrelation", FloatType(), False),
				StructField("productsWithBigCommentsQuantity", ArrayType(MapType(StringType(), IntegerType())), False)
            ])
		
		ch_product_df = spark.createDataFrame(data=[
				(products_highest_rate.select(array_agg(create_map("key", "value")).alias("data")).first()["data"],
				 product_lowest_rate.select(array_agg(create_map("key", "value")).alias("data")).first()["data"],
				 rate_and_sales_count_correlation.agg(corr("rating", "sales").alias("data")).first()["data"],
                 products_with_biggest_coments_quantity.select(array_agg(create_map("key", "value")).alias("data")).first()["data"])
		], schema=schema)
		
		ch_product_df2 = spark.createDataFrame(data=[
				Row(data=ch_product_df.toJSON().first())
		])
		
		ch_url, ch_props =	dbData["clickhouse"]
		
		ch_product_df2.write.jdbc(url=ch_url, table="quality_of_products", mode="append", properties=ch_props)

quantity_of_products_report()


Самые крутые товары


,key,value
0,Dog Food,5.0
1,Dog Food,5.0
2,Cat Toy,5.0
3,Cat Toy,5.0
4,Dog Food,5.0
5,Cat Toy,5.0
6,Bird Cage,5.0
7,Dog Food,5.0
8,Dog Food,5.0
9,Cat Toy,5.0


Самые плохие товары


,key,value
0,Bird Cage,1.0
1,Bird Cage,1.0
2,Dog Food,1.0
3,Bird Cage,1.0
4,Bird Cage,1.0
5,Cat Toy,1.0
6,Cat Toy,1.0
7,Cat Toy,1.0
8,Bird Cage,1.0
9,Cat Toy,1.0


Корреляция продаж и рейтинга(сырые данные)


,rating,sales
0,2.4,351
1,3.5,278
2,2.9,309
3,3.7,365
4,4.5,383


Товары с наибольшим количеством комментариев


,key,value
0,Bird Cage,1000
1,Bird Cage,1000
2,Dog Food,1000
3,Bird Cage,1000
4,Bird Cage,1000
5,Cat Toy,1000
6,Cat Toy,1000
7,Cat Toy,1000
8,Dog Food,1000
9,Bird Cage,1000


In [23]:
import json
from pprint import pprint

@spark_session
def show_inserted(spark):
    ch_url, ch_props = dbData["clickhouse"]
    for table in ["products", "sales_per_customers", "sales_per_times", "sales_per_stores", "sales_per_suppliers", "quality_of_products"
    ]:
        data_df = spark.read.jdbc(url=ch_url, table=table, properties=ch_props)
        data_df.createOrReplaceTempView(table)
        print(table)
        pprint(json.loads(spark.sql(f"select * from {table} limit 1;").first()["data"]))

show_inserted()

products
{'categoryRevenue': {'Cage': 217795.48, 'Food': 202240.47, 'Toy': 217933.8},
 'commentPerProduct': {'Bird Cage': 501.86755,
                       'Cat Toy': 506.5645,
                       'Dog Food': 495.1821},
 'mostRelevantProducts': ['Dog Food', 'Bird Cage', 'Cat Toy'],
 'ratePerProduct': {'Bird Cage': 3.0001493,
                    'Cat Toy': 3.00686,
                    'Dog Food': 3.0182989}}
sales_per_customers
{'customerCountryStratification': {'Afghanistan': 33,
                                   'Albania': 29,
                                   'American Samoa': 11,
                                   'Andorra': 11,
                                   'Argentina': 165,
                                   'Armenia': 27,
                                   'Australia': 2,
                                   'Austria': 1,
                                   'Azerbaijan': 19,
                                   'Bahamas': 16,
                                   'Bahrain': 1,
